In [8]:
from google.cloud import bigquery

client = bigquery.Client()

# Perform a query.
QUERY = (
    'SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` '
    'WHERE state = "TX" '
    'LIMIT 100')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.name)

Frances
Alice
Beatrice
Ella
Gertrude
Josephine
Lula
Blanche
Marjorie
Christine
Margarita
Alta
Sara
Naomi
Sofia
Andrea
Delfina
Dominga
Elnora
Adele
Rafaela
Dixie
Luisa
Bess
Ernestine
Lorine
Patsy
Cecelia
Enriqueta
Lucinda
Alyce
Oneta
Apolonia
Gloria
Rhoda
Buna
Maye
Pansy
Gladys
Maria
Helen
Velma
Clara
Anna
Evelyn
Rosa
Mae
Bernice
Stella
Vivian
Mable
Maggie
Betty
Guadalupe
Lorene
Susie
Sadie
Erma
Verna
Amelia
Estelle
Nell
Margie
Celia
Lessie
Eloise
Millie
Olga
Letha
Rachel
Luz
Gussie
Sylvia
Aline
Barbara
Iris
Leila
Mozelle
Simona
Catalina
Hester
Willia
Allene
Avis
George
Isabella
Polly
Syble
Teodora
Lennie
Ricarda
Valerie
Frances
Gertrude
Sarah
Rosa
Lucy
Nora
Nettie
Odessa


In [9]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [11]:
import pandas as pd

In [12]:
%%bigquery df
WITH
  gin_sales AS (
  SELECT
    DATE_TRUNC(date, MONTH) AS month,
    DATE_SUB(DATE_TRUNC(date, MONTH), INTERVAL 1 YEAR) AS month_year_before,
    SUM(volume_sold_liters) AS volume_sold_liters
  FROM
    `bigquery-public-data.iowa_liquor_sales.sales`
  WHERE
    UPPER(category_name) LIKE "%GIN%"
  GROUP BY
    month,
    month_year_before
  ORDER BY
    month DESC )
SELECT
  current_sales.month,
  current_sales.volume_sold_liters,
  year_ago_sales.volume_sold_liters AS volume_sold_year_ago,
  current_sales.volume_sold_liters - year_ago_sales.volume_sold_liters AS increase_liters
FROM
  gin_sales AS current_sales
JOIN
  gin_sales AS year_ago_sales
ON
  current_sales.month = year_ago_sales.month_year_before
ORDER BY
  1 DESC LIMIT 100; 

In [13]:
df.head()

,month,volume_sold_liters,volume_sold_year_ago,increase_liters
0,2019-03-01,36056.91,46507.25,-10450.34
1,2019-02-01,35150.41,35095.37,55.04
2,2019-01-01,41121.03,35968.30,5152.73
3,2018-12-01,49941.92,48926.89,1015.03
4,2018-11-01,42715.55,43390.38,-674.83


In [16]:
df = pd.read_gbq("""
WITH
  gin_sales AS (
  SELECT
    DATE_TRUNC(date, MONTH) AS month,
    DATE_SUB(DATE_TRUNC(date, MONTH), INTERVAL 1 YEAR) AS month_year_before,
    SUM(volume_sold_liters) AS volume_sold_liters
  FROM
    `bigquery-public-data.iowa_liquor_sales.sales`
  WHERE
    UPPER(category_name) LIKE "%GIN%"
  GROUP BY
    month,
    month_year_before
  ORDER BY
    month DESC )
SELECT
  current_sales.month,
  current_sales.volume_sold_liters,
  year_ago_sales.volume_sold_liters AS volume_sold_year_ago,
  current_sales.volume_sold_liters - year_ago_sales.volume_sold_liters AS increase_liters
FROM
  gin_sales AS current_sales
JOIN
  gin_sales AS year_ago_sales
ON
  current_sales.month = year_ago_sales.month_year_before
ORDER BY
  1 DESC LIMIT 100; 
""", use_bqstorage_api=True)

Downloading: 100%|██████████| 87/87 [00:03<00:00, 28.32rows/s]


In [17]:
df.to_gbq("experimental.test_table", "bigquery-course-xyz", if_exists='replace')

1it [00:05,  5.02s/it]
